In [ ]:
from matplotlib import pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df=pd.read_csv('drive/My Drive/churn-5000.csv')

In [ ]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)


churn=cat_df['churn']
cat_df=cat_df.drop(['churn'], axis=1)



num_df=num_df.fillna(conData)

result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)

result_df_op=result_df
#result_df_op=result_df_op.drop(['churn'], axis=1)
#result_df_op=np.nan_to_num(result_df_op)


result_df_op=result_df

X=result_df_op
y=churn

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
train=pd.concat([X_train, y_train], axis=1)
test=pd.concat([X_test, y_test], axis=1)
train.to_csv("drive/My Drive/Original_data_GAN_mWOE_privacy_1_70percent_training_dataseat5000.csv")
test.to_csv("drive/My Drive/Original_data_GAN_mWOE_privacy_1_30percent_testing_dataseat5000.csv")
#train=pd.concat([X, y], axis=1)
#test=pd.concat([X, y], axis=1)
#X_train=X
#X_test=X
#y_train=y
#y_test=y

In [ ]:
from google.colab import files
import os, sys
file_path = 'drive/My Drive/Colab Notebooks/private-data-generation/'
#sys.path.append(file_path)
sys.path.append(os.path.abspath(file_path))
import sys
sys.path.insert(1,'/content/drive/My Drive/Colab Notebooks/private-data-generation/')

In [ ]:
from models import dp_wgan, pate_gan, ron_gauss

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn import preprocessing
from scipy.special import expit
from models import dp_wgan, pate_gan, ron_gauss
from models.Private_PGM import private_pgm
import argparse
import numpy as np
import pandas as pd
import collections
import os

In [ ]:
class  Class_opt:
    "This is a person class"
    categorical = False
    model='dp-wgan'
    target_epsilon=80
    target_delta=0.8
    downstream_task="classification"
    target_variable='status'
    batch_size=64
    micro_batch_size=8
    clamp_lower=0.01
    clamp_upper=0.01
    clip_coeff=0.1
    sigma=2.0
    num_epochs=50
    enable_privacy=0




opt = Class_opt()
opt.target_variable='churn'
opt.categorical = True
opt.model='dp-wgan'
opt.target_epsilon=10
opt.target_delta=0.1
opt.batch_size=64
opt.micro_batch_size=8
opt.clamp_lower=0.01
opt.clamp_upper=0.01
opt.clip_coeff=0.1
opt.sigma=0.8
opt.num_epochs=200
opt.enable_privacy=1

opt.downstream_task="classification"

In [ ]:
data_columns = [col for col in train.columns if col != opt.target_variable]

In [ ]:
#import numpy as np
#class_ratios = train[opt.target_variable].sort_values().groupby(train[opt.target_variable]).size().values/train.shape[0]
class_ratios = np.array([0.50, 0.50])
class_ratios

array([0.5, 0.5])

In [ ]:
X_train = np.nan_to_num(train.drop([opt.target_variable], axis=1).values)
y_train = np.nan_to_num(train[opt.target_variable].values)
X_test = np.nan_to_num(test.drop([opt.target_variable], axis=1).values)
y_test = np.nan_to_num(test[opt.target_variable].values)

In [ ]:
#Normalized the data
X_train = expit(X_train)
X_test = expit(X_test)

In [ ]:
input_dim = X_train.shape[1]
z_dim = int(input_dim / 4 + 1) if input_dim % 4 == 0 else int(input_dim / 4)

Hyperparams = collections.namedtuple('Hyperarams','batch_size micro_batch_size clamp_lower clamp_upper clip_coeff sigma class_ratios lr num_epochs')
Hyperparams.__new__.__defaults__ = (None, None, None, None, None, None, None, None, None)

In [ ]:
conditional = (opt.downstream_task == "classification")
model = dp_wgan.DP_WGAN(input_dim, z_dim, opt.target_epsilon, opt.target_delta, conditional)
model.train(X_train, y_train, Hyperparams(batch_size=opt.batch_size, micro_batch_size=opt.micro_batch_size,
                                              clamp_lower=opt.clamp_lower, clamp_upper=opt.clamp_upper,
                                              clip_coeff=opt.clip_coeff, sigma=opt.sigma, class_ratios=class_ratios, lr=
                                              5e-5, num_epochs=opt.num_epochs), private=opt.enable_privacy)

torch.save(model, 'drive/My Drive/chk_p_GAN_MWOE_privacy_1_dataset5000_70-vs-30.pth')

Epoch : 1 Loss D real :  0.026751828047378502 Loss D fake :  0.011735967956451408 Loss G :  0.011893228816460141 Epsilon spent :  3.3807596723163544
Epoch : 2 Loss D real :  0.026619757848717514 Loss D fake :  0.011665658519264887 Loss G :  0.011808905266267124 Epsilon spent :  3.8832879783901513
Epoch : 3 Loss D real :  0.026794563869527284 Loss D fake :  0.011729568182367898 Loss G :  0.011652704277303493 Epsilon spent :  4.3858162844639486
Epoch : 4 Loss D real :  0.026712702818177603 Loss D fake :  0.011622303500197526 Loss G :  0.01179983657699264 Epsilon spent :  4.6538087602691025
Epoch : 5 Loss D real :  0.02682302849951374 Loss D fake :  0.011542134196995048 Loss G :  0.011654703044354914 Epsilon spent :  4.857850494922193
Epoch : 6 Loss D real :  0.02661684330589518 Loss D fake :  0.011522418989524231 Loss G :  0.011806247230655252 Epsilon spent :  5.061892229575283
Epoch : 7 Loss D real :  0.02679023972864777 Loss D fake :  0.011534853682768449 Loss G :  0.011641174154366855

In [ ]:
if opt.model == 'imle' or opt.model == 'dp-wgan' or opt.model == 'pate-gan':
  syn_data = model.generate(X_train.shape[0], class_ratios)
  X_syn, y_syn = syn_data[:, :-1], syn_data[:, -1]

X_syn_df = pd.DataFrame(data=X_syn, columns=data_columns)
y_syn_df = pd.DataFrame(data=y_syn, columns=[opt.target_variable])

syn_df = pd.concat([X_syn_df, y_syn_df], axis=1)
syn_df.to_csv("drive/My Drive/synthetic_data_GAN_mWOE_privacy_1_70percent_training_dataset5000_set_15.csv")
print("Saved synthetic data at : ", "drive/My Drive/synthetic_data_GAN_mWOE_privacy_1_70percent_training_dataset5000.csv")

Saved synthetic data at :  drive/My Drive/synthetic_data_GAN_mWOE_privacy_1_70percent_training_dataset5000.csv
